# PART 2 OF CODING - GATHERING OF DETAILS OF RESTAURANTS LISTED IN PART 1

In [1]:
%reset -f
import gc
gc.collect()

import pandas as pd
import numpy as np
from IPython.display import display, HTML
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests

print('Libraries imported.')

Libraries imported.


## Importing file 'export_dataframe1.csv' created and stored in Part 1

In [2]:
ny_ff = pd.read_csv('export_dataframe1.csv')
print(ny_ff.shape)
display(HTML(ny_ff.head().to_html()))

(173, 6)


,zip,Latitude,Longitude,Name,Id,Venue Category
0,10001,40.748724,-74.003422,SUBWAY,4ecbe682f9f4a82587658269,Sandwich Place
1,10001,40.747569,-73.997088,McDonald's,4b1331eef964a5205c9523e3,Fast Food Restaurant
2,10001,40.752250,-74.005727,Enfes,59c143aa6cf01a3c2418005a,Turkish Restaurant
3,10001,40.748357,-74.004128,Subway,5a64e13cea1e445448e69273,Sandwich Place
4,10001,40.746259,-73.997676,Subway,4d6e842538363704f4d8a9d0,Sandwich Place


## Seeing shape and data of list of restaurants, uniquely identified by Ids

In [3]:
print(ny_ff.shape)
display(HTML(ny_ff.head().to_html()))
#print(ny_ff.groupby('zip').count())
#print(ny_ff.groupby('Id').count())

(173, 6)


,zip,Latitude,Longitude,Name,Id,Venue Category
0,10001,40.748724,-74.003422,SUBWAY,4ecbe682f9f4a82587658269,Sandwich Place
1,10001,40.747569,-73.997088,McDonald's,4b1331eef964a5205c9523e3,Fast Food Restaurant
2,10001,40.752250,-74.005727,Enfes,59c143aa6cf01a3c2418005a,Turkish Restaurant
3,10001,40.748357,-74.004128,Subway,5a64e13cea1e445448e69273,Sandwich Place
4,10001,40.746259,-73.997676,Subway,4d6e842538363704f4d8a9d0,Sandwich Place


## Setting up Foursquare Credentials

In [42]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


## Retrieving details of first restaurant and studying the structure of the JSON file

In [6]:
id=ny_ff.loc[0, 'Id']
id='589a09023bd4ab658ea355f3'
neighbourhood_latitude = ny_ff.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = ny_ff.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = ny_ff.loc[0, 'Name'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))
url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
)
          
print(url)
#results = requests.get(url).json()
#results

Latitude and longitude values of SUBWAY are 40.74872402803528, -74.00342170522295.
https://api.foursquare.com/v2/venues/589a09023bd4ab658ea355f3?&client_id=&client_secret=&v=20180605


## Looking at its count of likes, price tier and rating

In [7]:
print(results['response']['venue']['likes']['count'])
print(results['response']['venue']['price']['tier'])
print(results['response']['venue']['rating'])

NameError: name 'results' is not defined

## Defining a function to extract details of all restaurants for each zip code by Restaurant Ids

In [43]:
def getRatings(RestaurantIds):
    
    ratings=[]
    for id in RestaurantIds:
        #print(id)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
        )
        print(url)
        # make the GET request
        results = requests.get(url).json()
        #print(results)
        # return only relevant information for each nearby venue
        try:
            count=results['response']['venue']['likes']['count']
        except:
            count=0
        try:
            priceTier=results['response']['venue']['price']['tier']
        except:
            priceTier=0
        try:
            rating=results['response']['venue']['rating']
        except:
            rating=0
        ratings.append([
            id, 
            count,
            priceTier,
            rating
        ])

    #df_ratings = pd.DataFrame([item for ratings in ratings for item in ratings])
    #df_ratings = pd.DataFrame([item for ratings in ratings for item in ratings])
    df_ratings=pd.DataFrame(ratings)
    df_ratings.columns = ['RestaurantId','Likes', 'Price_Tier','Ratings']
    print('Ratings Extracted!!')
    return(df_ratings)

## Callling the function for Restaurant Ids listed in 'export_dataframe1.csv' file

In [45]:
lastIndex=len(ny_ff['Id'])
RestaurantIds=(ny_ff.iloc[149:lastIndex,4])
#RestaurantIds=['5b720467037be1002c11b69b']

print(RestaurantIds.count())
print(RestaurantIds)

df_ratings = getRatings(RestaurantIds)

print('Data saved and exported!!')

24
149    4bf1f678d39ad13a9ce5aa0e
150    4d22633b04f559412373f585
151    51382eece4b0cdf571c3956c
152    51b79ea887f367858131aa5b
153    4b25f542f964a520c47724e3
154    4bbd288407809521d950da91
155    55a15550498e4dc36c4845c3
156    4acfaa7cf964a52032d520e3
157    4de15acde4cd056f7429fad9
158    4ac902a2f964a52003be20e3
159    563ee2e4cd10872628b2cf66
160    4b074da2f964a5209bfb22e3
161    4e4ce4d1bd413c4cc66d0562
162    4c1ba521eac020a1382d45c2
163    4bc7cd216501c9b63e3b3f29
164    516080f4e4b0dcf88a609cf1
165    4d1a4cf51356a0934d7ff082
166    4bf1d250d39ad13a85b4aa0e
167    4b19debcf964a5207ce523e3
168    4be84f89c5220f472d95a9ca
169    4a0c9f36f964a5204e751fe3
170    4c9dbaab7ada199cc4b794bc
171    4ba23faef964a520f2e637e3
172    4e4ce455bd413c4cc66d0138
Name: Id, dtype: object
https://api.foursquare.com/v2/venues/4bf1f678d39ad13a9ce5aa0e?&client_id=&client_secret=&v=20180605
https://api.foursquare.com/v2/venues/4d22633b04f559412373f585?&client_id=&client_secret=&v=20180605
https

## Looking for data in extracted dataframe

In [48]:
df_ratings.head()


,RestaurantId,Likes,Price_Tier,Ratings
0,4bf1f678d39ad13a9ce5aa0e,22,1,5.8
1,4d22633b04f559412373f585,1,1,6.7
2,51382eece4b0cdf571c3956c,0,1,5.7
3,51b79ea887f367858131aa5b,2,1,5.8
4,4b25f542f964a520c47724e3,14,1,5.1


In [49]:
df_ratings.drop_duplicates(subset=['RestaurantId'], keep="first",inplace=True)
df_ratings.reset_index()
print(df_ratings.shape)
print(df_ratings.head())
print(df_ratings.groupby('RestaurantId').count())

(24, 4)
               RestaurantId  Likes  Price_Tier  Ratings
0  4bf1f678d39ad13a9ce5aa0e     22           1      5.8
1  4d22633b04f559412373f585      1           1      6.7
2  51382eece4b0cdf571c3956c      0           1      5.7
3  51b79ea887f367858131aa5b      2           1      5.8
4  4b25f542f964a520c47724e3     14           1      5.1
                          Likes  Price_Tier  Ratings
RestaurantId                                        
4a0c9f36f964a5204e751fe3      1           1        1
4ac902a2f964a52003be20e3      1           1        1
4acfaa7cf964a52032d520e3      1           1        1
4b074da2f964a5209bfb22e3      1           1        1
4b19debcf964a5207ce523e3      1           1        1
4b25f542f964a520c47724e3      1           1        1
4ba23faef964a520f2e637e3      1           1        1
4bbd288407809521d950da91      1           1        1
4bc7cd216501c9b63e3b3f29      1           1        1
4be84f89c5220f472d95a9ca      1           1        1
4bf1d250d39ad13a85b4

## Due to limited Foursquare premium calls, saving this information in csv file 'export_dataframe2.csv'

In [50]:
import base64
from IPython.display import HTML

def create_download_link( df_ratings, title = "Download CSV file", filename = "export_dataframe2.csv"):  
    csv = df_ratings.to_csv(index = None, header=True)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_ratings)